In [124]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from PIL import Image

In [2]:
# GLOBAL PATH VARIABLE FOR AUDIO / TXT FILES:
path = 'respiratory-sound-database/audio_and_txt_files/'

In [3]:
def get_audio_files():
    folder = os.listdir(path)

    wav_files, txt_files = [], []
    for file in folder:
        if file.endswith('_16.wav'):
            wav_files.append(file)
        elif file.endswith('.txt'):
            txt_files.append(file)

    return wav_files, txt_files

In [4]:
def extract_info(filename):
    splits = filename.split('_')
    print(splits)

In [5]:
# separate files by audio and txt
wav_files, txt_files = get_audio_files()
wav_files = sorted(wav_files)
txt_files = sorted(txt_files)

In [6]:
# extract information from the wav files' filename
# 0:[patient #], 1:[recording index], 2:[chest location], 3:[acquisition mode], 4:[recording equipment]
stripped_file_info = [re.sub('_16\.wav$', '', file) for file in wav_files]
file_info = [file.split('_') for file in stripped_file_info]

In [7]:
# load the extracted data into a dataframe
demog_data = pd.read_csv('demographic-info.csv')
diag_data = pd.read_csv('respiratory-sound-database/patient_diagnosis.csv', names=['Patient Number', 'Diagnosis'])
data = pd.DataFrame(data=file_info, columns=['Patient Number', 'Recording Index', 'Chest Location', 'Acquisition Mode', 'Recording Equipment'])
data['Patient Number'] = data['Patient Number'].astype(int)
data['txt_file'] = txt_files
data['audio_file'] = wav_files

In [8]:
# put info from demog_data into data
age, sex, a_BMI, c_weight, c_height, diagnosis = [], [], [], [], [], []
demog_size = demog_data['Age'].size
size = data['Patient Number'].size

In [9]:
# TODO: Make this more efficient
for j in range(0, size):
    for i in range(0, demog_size):
        if data['Patient Number'][j] == demog_data['Patient Number'][i]:
            age.append(demog_data['Age'][i])
            sex.append(demog_data['Sex'][i])
            a_BMI.append(demog_data['Adult BMI'][i])
            c_weight.append(demog_data['Child Weight'][i])
            c_height.append(demog_data['Child Height'][i])
        if data['Patient Number'][j] == diag_data['Patient Number'][i]:
            diagnosis.append(diag_data['Diagnosis'][i])
        pass

In [10]:
data['Age'], data['Sex'], data['Adult BMI'], data['Child Weight'], data['Child Height'], data['Diagnosis'] = age, sex, a_BMI, c_weight, c_height, diagnosis

In [11]:
# function to get spectrogram from a wav file
def get_spectro(file_path):
    sample_rate, samples = wavfile.read(path + file_path)
    Pxx, freqs, bins, im = plt.specgram(samples[:], NFFT=1024, Fs=44100, noverlap=900)

    save_path = path + 'spectrograms/' + re.sub('\.wav$', '_spec', file_path)
    plt.savefig(save_path)

In [75]:
spectrograms = [f for f in os.listdir(path + 'spectrograms/')]
spectrograms.remove('.DS_Store')
spectrograms.sort()

data['spec_file'] = spectrograms

In [88]:
# train split
# Notes: I'm splitting this manually so that the data is "speaker" independent.
# Important: the range of patient numbers is 101 - 226.
end = 226
beg = 101
# train_ind is the SPEAKER index that signifies the end of the training set. testing and val will have other speaker data in them.
train_ind = np.floor(0.7 * (end - beg) + beg)
test_ind = np.floor(0.5 * (end - train_ind) + train_ind)
val_ind = 226

# find row index where train_ind ends
row = 0
for i in range(0, len(data.index)):
    if data['Patient Number'][i] == train_ind:
        row = i

train_data = data.loc[:row, :]

In [91]:
# test split
test_row = [i for i in range(row, len(data.index)) if data['Patient Number'][i] == test_ind]
test_data = data.loc[row + 1:test_row[-1], :]

In [94]:
val_row = [i for i in range(row, len(data.index)) if data['Patient Number'][i] == val_ind]
val_data = data.loc[test_row[-1] + 1:, :]

In [138]:
X_train, y_train = train_data['spec_file'], train_data['Diagnosis']
X_test, y_test = test_data['spec_file'], test_data['Diagnosis']

In [139]:
image = Image.open(path + 'spectrograms/' + X_train[0])

# summarize some details about the image
# print(image.format)
# print(image.size)
# print(image.mode)
# show the image
# image.show()

ValueError: not a bitmap

In [133]:
from keras.datasets import mnist
#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

In [2]:
# TODO: @Priya
# 2. Look into training CNN on spectrograms
# 3. Train CNN on spectrograms